The main workspace for calculating pseudoinverse $A^\dagger$ of nonsymmetric matrices

In [ ]:
workspace()

In [86]:
using Plots
using MatrixDepot   # for downloading and loading matrices from the UF sparse matrix collection
using StatsBase
using JLD  
include("../src/Pseudoinverse.jl")
importall Pseudoinverse

In [72]:
# Uniform random
m = 500;    # number of columns
n= 1000;    # number of rows
r = 200;    # rankd
prob = uniform_mat_rank(m,n,r);

In [90]:
# Rank deficient matrices from UF sparse matrix collection
# NYPA/Maragal_3, NYPA/Maragal_4, NYPA/Maragal_5, 
# Meszaros/primagaz  LPnetlib/lp_fit2d  LPnetlib/lp_ken_07
name = "NYPA/Maragal_3"  
#matrixdepot(name, :get);    # Use this for first time call
A = matrixdepot(name, :read);
exacterror =false;
sA= size(A); m= sA[1] ; n = sA[2];
prob = Prob(A,[],[], [], name);
size(A)

(1690,860)

In [91]:
#Basic parameters
tol = 10.0^(-2); restol = 10.0^(-5);
maxiter=1000;
max_time = 20;
printiters = true;
exacterror =false;

In [92]:
skip_error_calculation = 1;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))
options = MyOptions(n,m,0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror, "none") 
OUTPUTS = [];

In [93]:
# Testing methods

method_name = "SATAX";#"SATAX"; "NewtonSchultz" "SAXAS"
options.sketch = "uni";
options.M0type = "ATproj";   # Select initial matrix, options: ATproj, eye
output= pseudoinvert_matrix(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

options.sketch = "ada";
options.M0type = "ATproj";
output= pseudoinvert_matrix(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];


method_name = "NewtonSchulz";
output= pseudoinvert_matrix(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

# Warm starting the NewtonSchulz method using the SATAX method.
method_name = "NewtonSchulz_warm";
options.sketch = "ada";
options.M0type = "ATproj";
output= pseudoinvert_matrix(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];


SATAX-uni
-------------------
It   | Error% | Residual |  Time   
-------------------
  1  | 100.00  |  52.82  | 0.0790 
  2  | 100.00  |  38.18  | 0.1734 
  3  | 100.00  |  34.19  | 0.2780 
  4  | 100.00  |  32.59  | 0.3519 
  5  | 100.00  |  30.79  | 0.4127 
  6  | 100.00  |  28.53  | 0.4751 
  7  | 100.00  |  27.04  | 0.5712 
  8  | 100.00  |  25.40  | 0.6950 
  9  | 100.00  |  23.85  | 0.7874 
 10  | 100.00  |  22.86  | 0.8979 
 11  | 100.00  |  22.45  | 1.0175 
 12  | 100.00  |  20.38  | 1.0937 
 13  | 100.00  |  19.15  | 1.2157 
 14  | 100.00  |  16.76  | 1.3420 
 15  | 100.00  |  16.35  | 1.4534 
 16  | 100.00  |  15.50  | 1.5984 
 17  | 100.00  |  15.03  | 1.6693 
 18  | 100.00  |  14.64  | 1.7395 
 19  | 100.00  |  14.31  | 1.8442 
 20  | 100.00  |  13.65  | 1.9228 
 21  | 100.00  |  13.29  | 1.9903 
 22  | 100.00  |  12.73  | 2.1119 
 23  | 100.00  |  12.36  | 2.2159 
 24  | 100.00  |  12.33  | 2.3605 
 25  | 100.00  |  11.98  | 2.4365 
 26  | 100.00  |  11.66  | 2.5495 
 27 

In [94]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename).jld", "OUTPUTS", OUTPUTS)

In [95]:
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

LoadError: LoadError: UndefVarError: pgfplots not defined
while loading In[95], in expression starting on line 1

In [3]:
# Loading saved problem data
probname = "rcv1_train.binary-ridge"#"gisette_scale-ridge";  
test = load("../data/$(probname).jld")
OUTPUTS = test["OUTPUTS"];
# mask = [1,3,5]
# OUTPUTS[mask]

In [96]:
savename

"NYPA-Maragal_3"